<a href="https://colab.research.google.com/github/cedro3/others2/blob/main/video_matting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Robust High-Resolution Video Matting with Temporal Guidance.

![Teaser](https://raw.githubusercontent.com/PeterL1n/RobustVideoMatting/master/documentation/image/teaser.gif)

[Project Site](https://peterl1n.github.io/RobustVideoMatting) | [GitHub](https://github.com/PeterL1n/RobustVideoMatting) | [Paper](https://arxiv.org/abs/2108.11515)


## セットアップ（３分程度かかります）

In [ ]:
# pytorchバージョン変更
! pip install torch==1.8.0 torchvision==0.9.0

# ライブラリー・インストール
! pip install --quiet av pims

# モデル構築
import torch
model = torch.hub.load("PeterL1n/RobustVideoMatting", "mobilenetv3").cuda() # or "resnet50"
convert_video = torch.hub.load("PeterL1n/RobustVideoMatting", "converter")

### オプション 1: 自分の動画のアップロード

セルを実行して自分のPCにあるファイルを選択 (mp4ファイルのみ)\
＊アップロードしたファイルは自動的にinput.mp4に書き換えられます

In [ ]:
import os
from google.colab import files

uploaded = files.upload()           # Use colab upload dialog.
uploaded = list(uploaded.keys())    # Get uploaded filenames.
assert len(uploaded) == 1           # Make sure only uploaded one file.
os.rename(uploaded[0], 'input.mp4') # Rename file to "input.mp4".

### オプション 2: サンプルビデオを使用（自分のビデオを使う場合はパス）

セルを実行してサンプルビデオをダウンロードする\
＊ダウンロードしたファイルは自動的にinput.mp4に書き換えられます

In [ ]:
import gdown
gdown.download('https://drive.google.com/uc?id=1LASEXzU015e30y4YQqv6mq4KId07mVof', './input.mp4', quiet=False)

### 変換の実行

In [ ]:
convert_video(
    model,                           # The loaded model, can be on any device (cpu or cuda).
    input_source='input.mp4',        # A video file or an image sequence directory.
    downsample_ratio=None,           # [Optional] If None, make downsampled max size be 512px.
    output_type='video',             # Choose "video" or "png_sequence"
    output_composition='com.mp4',    # File path if video; directory path if png sequence.
    output_alpha="pha.mp4",          # [Optional] Output the raw alpha prediction.
    output_foreground="fgr.mp4",     # [Optional] Output the raw foreground prediction.
    output_video_mbps=4,             # Output video mbps. Not needed for png sequence.
    seq_chunk=12,                    # Process n frames at once for better parallelism.
    num_workers=1,                   # Only for image sequence input. Reader threads.
    progress=True                    # Print conversion progress.
)

### 動画の再生

In [ ]:
# mp4動画の再生
from IPython.display import HTML
from base64 import b64encode

mp4 = open('./com.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="70%" height="70%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

### 実行結果

変換を実行すると下記のファイルが作成されます。必要に応じてダウンロードして下さい。

* `com.mp4`: 【背景をグリーンスクリーン化した動画】
* `pha.mp4`: αチャンネルを操作した動画.
* `fgr.mp4`: マスク動画